In [23]:
import random
import pandas as pd
import numpy as np
import cvxpy as cp
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc('font', family='BIZ UDGothic')

In [71]:
# 行動分類のラベル
large = ["必需行動", "拘束行動", "自由行動", "その他"] # 大分類
 
middle = ["睡眠", "食事", "身のまわりの用事", "療養・静養", "仕事", "仕事関連", "学業", "学校外の学習", 
  "家事", "子どもの世話", "家庭雑事", "通勤", "通学", "社会参加", "会話・交際", "スポーツ", 
  "行楽・散策", "趣味・娯楽・教養 (インターネット除く)", "趣味・娯楽・教養のインターネット (動画除く)", 
  "インターネット動画", "テレビ", "録画番組・DVD", "ラジオ", "新聞", "雑誌・マンガ・本", "音楽", "休息", "その他", "不明"]
 # 中分類
 
small = ["睡眠", "食事", "身のまわりの用事", "療養・静養", "仕事", "仕事のつきあい", "授業", "学校外の学習", 
  "炊事・掃除・洗濯", "買い物", "子どもの世話", "家庭雑事", "通勤", "通学", "社会参加", "会話・交際", 
  "スポーツ", "行楽・散策", "趣味・娯楽・教養 (インターネット除く)", "趣味・娯楽・教養のインターネット (動画除く)", 
  "インターネット動画", "テレビ", "録画番組・DVD", "ラジオ", "新聞", "雑誌・マンガ・本", "音楽", "休息", "その他", "不明"]
 # 小分類

def calc_list_diff(list1:list, list2:list):
    # リストの長さが異なる場合のエラーチェック
    if len(list1) != len(list2):
        raise ValueError("リストの長さが異なります。")
    
    # 各要素の差分を計算して新しいリストを生成
    difference = [a - b for a, b in zip(list1, list2)]
    return difference

# not P_iの計算
def get_P_i(Initial_P_i,Transition_rates,Daily_table):
    initial_rate = np.array(Initial_P_i)

    # 正規化
    initial_rate = initial_rate / np.sum(initial_rate)

    P_i_list = []
    for i in range(len(Daily_table)):

        # 行列の加工
        n = len(initial_rate)
        one_vector = np.ones(n)
        modified_transition_rates = []

        for A in Transition_rates:
            A_i = A.copy()
            A_i[i, :] = 0  # 行 i を全て 0 に置き換える
            modified_transition_rates.append(A_i)

        # 初期行為者率ベクトルの加工
        modified_initial_rate = initial_rate.copy()
        modified_initial_rate[i] = 0

        # 最終的な行為者率ベクトルの計算
        final_rate = modified_initial_rate

        for A_i in modified_transition_rates:
            final_rate = A_i @ final_rate

        # 行為 i を一度も行わない確率の計算
        P_not_i = one_vector @ final_rate

        P_i = 1-P_not_i

        P_i_list.append(P_i)

    return P_i_list

# データフレームを参照リストに基づいてソートするヘルパー関数
def sort_df_by_list(df, column, reference_list):
    return df.set_index(column).reindex(reference_list).reset_index()

# 遷移率を計算する関数
def calc_transition_rates(beta_i: np.ndarray, time_list: list, time_table: pd.DataFrame, activity_list: list):
    transition_rates = []
    middle = activity_list  # 活動のリストを基準として使用
    
    for i in range(len(time_list)):
        time_t = time_list[i]
        time_t1 = time_list[(i+1) % len(time_list)]  # 次の時間帯
        
        # 両方の時間帯の行動率を取得
        y_t = np.array(sort_df_by_list(time_table[(time_table["Time"].dt.hour == time_t.hour) & (time_table["Time"].dt.minute == time_t.minute) ], "Activity", middle)["Rate"])
        y_t1 = np.array(sort_df_by_list(time_table[(time_table["Time"].dt.hour == time_t1.hour) & (time_table["Time"].dt.minute == time_t1.minute) ], "Activity", middle)["Rate"])
        
        # 変数の定義
        no_of_elem = len(middle)
        a = cp.Variable((no_of_elem, no_of_elem), nonneg=True)  # 非負の要素を持つ遷移行列

        # 目的関数と制約条件の定義
        delta = np.eye(no_of_elem)  # クロネッカーのデルタ
        beta_i = np.ones(no_of_elem)
        # 目的関数: (1 - beta_i * delta) で修正された、二乗差の合計を最小化
        objective = cp.Minimize(cp.sum(cp.multiply(1 - beta_i * delta, cp.square(a))))

        # 行の和が1になる制約に変更
        constraints = [
            cp.sum(a, axis=1) == 1,  # 遷移行列の各行の和が1になる制約
            a >= 0,  # 非負制約
        ]

        # 行動率を正規化
        y_t = y_t / np.sum(y_t)
        y_t1 = y_t1 / np.sum(y_t1)

        # 次の時間帯の行動率と遷移行列の適用が一致する制約
        constraints.append(y_t1 - a.T @ y_t == 0)

        # 問題を定義する
        problem = cp.Problem(objective, constraints)

        # ソルバーを使用して問題を解く（別のソルバーを指定して解く）
        problem.solve(solver=cp.SCS, verbose=True)

        # 得られた遷移行列を格納
        transition_rates.append(a.value)

    return np.array(transition_rates)

def roulette_wheel_selection(weights):
    """
    Performs roulette wheel selection on a list of weights.
    
    Args:
    weights (list of float): The weights or probabilities for each item.
    
    Returns:
    int: The index of the selected item.
    """
    # Calculate the cumulative sum of weights
    cumulative_sum = [sum(weights[:i+1]) for i in range(len(weights))]
    total_sum = cumulative_sum[-1]
    
    # Generate a random number in the range [0, total_sum)
    random_num = random.uniform(0, total_sum)
    
    # Find the index where the random number would fit in the cumulative sum
    for i, cum_sum in enumerate(cumulative_sum):
        if random_num < cum_sum:
            return i

In [72]:
data_hourly = pd.read_csv('../data/2020_jikoku_danjonenso.csv')
data_full_day = pd.read_csv('../data/2020_4shihyo_danjonenso.csv')

In [101]:
data_full_day

,曜日,層,行動,行為者率(％),行為者平均時間量(時間:分),全員平均時間量(時間:分),全員平均時間量の標準偏差(時間:分)
0,平日,男１０代,睡眠,98.9,7:40,7:35,1:24
1,平日,男２０代,睡眠,100.0,7:27,7:27,1:14
2,平日,男３０代,睡眠,99.5,7:23,7:21,1:44
3,平日,男４０代,睡眠,99.7,6:59,6:58,1:38
4,平日,男５０代,睡眠,99.4,6:56,6:54,1:32
...,...,...,...,...,...,...,...
1801,日曜日,女３０代,テレビ（専 念）,44.8,1:50,0:49,1:25
1802,日曜日,女４０代,テレビ（専 念）,60.0,2:09,1:18,1:42
1803,日曜日,女５０代,テレビ（専 念）,81.5,2:45,2:15,2:10
1804,日曜日,女６０代,テレビ（専 念）,89.4,3:19,2:58,2:33


In [85]:
data_hourly = data_hourly.set_axis(["Day","Group","Activity","Time","Rate"],axis=1)
data_hourly["Time"] = pd.to_datetime(data_hourly["Time"], format="%H:%M")
data_hourly = data_hourly.query("Group == '男１０代' & Activity in @middle & Day == '平日'")
data_hourly_cleaned = data_hourly[["Activity","Time","Rate"]].copy()
data_hourly_cleaned = data_hourly_cleaned.sort_values(by="Time")

# アクティビティと時間のリストを取得
activity_list_hourly = data_hourly_cleaned['Activity'].unique()
time_list_hourly = data_hourly_cleaned['Time'].unique()

                                     CVXPY                                     
                                     v1.5.2                                    
(CVXPY) Sep 09 03:35:02 PM: Your problem has 576 variables, 624 constraints, and 0 parameters.
(CVXPY) Sep 09 03:35:02 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Sep 09 03:35:02 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Sep 09 03:35:02 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Sep 09 03:35:02 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Sep 09 03:35:02 PM: Compiling problem (target solver=SCS).
(C

c:\Users\tora2\anaconda3\envs\geo_env\Lib\site-packages\cvxpy\problems\problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


  4000| 2.03e-04  8.00e-07  2.49e-05  1.76e-02  1.00e-01  1.76e-01 
  4250| 3.91e-05  7.13e-07  2.67e-05  1.77e-02  1.00e-01  1.88e-01 
  4500| 3.74e-05  3.68e-07  1.55e-05  1.77e-02  1.00e-01  2.00e-01 
  4750| 3.59e-05  2.49e-07  9.88e-06  1.77e-02  1.00e-01  2.13e-01 
  4950| 1.88e-07  7.58e-08  2.14e-07  1.77e-02  1.00e-01  2.22e-01 
------------------------------------------------------------------
status:  solved
timings: total: 2.22e-01s = setup: 3.63e-03s + solve: 2.18e-01s
	 lin-sys: 1.33e-01s, cones: 2.93e-02s, accel: 9.89e-03s
------------------------------------------------------------------
objective = 0.017653
------------------------------------------------------------------
-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
(CVXPY) Sep 09 03:35:10 PM: Problem status: optimal
(CVXPY) S

可視化

In [90]:
# ヒートマップを描画する関数
def plot_transition_heatmaps(transition_rates, activity_list, time_list):
    num_time_intervals = len(time_list)
    num_activities = len(activity_list)

    # 時間ごとにヒートマップを描画
    fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(20, 20))
    axes = axes.flatten()  # 5x5のグリッドにして1次元に展開

    for i in range(min(num_time_intervals, 25)):  # 最大25時間分描画
        ax = axes[i]
        sns.heatmap(transition_rates[i], annot=False, cmap='coolwarm', cbar=True, vmin=0, vmax=1, ax=ax)
        ax.set_title(f"{i}-{(i+1)%24}時", fontsize=10)
        ax.set_xticks(np.arange(len(activity_list)) + 0.5)
        ax.set_yticks(np.arange(len(activity_list)) + 0.5)
        ax.set_xticklabels(activity_list, rotation=90, fontsize=8)
        ax.set_yticklabels(activity_list, rotation=0, fontsize=8)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# 先ほど計算した transition_rates_hourly を使用してヒートマップを作成
plot_transition_heatmaps(transition_rates_hourly, activity_list_hourly, time_list_hourly)

In [14]:
import random

In [93]:
def roulette_wheel_selection(weights):
    """
    Performs roulette wheel selection on a list of weights.
    
    Args:
    weights (list of float): The weights or probabilities for each item.
    
    Returns:
    int: The index of the selected item.
    """
    # Calculate the cumulative sum of weights
    cumulative_sum = [sum(weights[:i+1]) for i in range(len(weights))]
    total_sum = cumulative_sum[-1]
    
    # Generate a random number in the range [0, total_sum)
    random_num = random.uniform(0, total_sum)
    
    # Find the index where the random number would fit in the cumulative sum
    for i, cum_sum in enumerate(cumulative_sum):
        if random_num < cum_sum:
            return i

In [94]:
activity_list_hourly

array(['睡眠', '休息', '療養・静養', '仕事関連', '仕事', '音楽', '学業', '学校外の学習', '家事',
       '雑誌・マンガ・本', '身のまわりの用事', '子どもの世話', '通勤', '新聞', '通学', '社会参加',
       '会話・交際', 'ラジオ', 'スポーツ', '行楽・散策', 'インターネット動画', '家庭雑事', '食事', 'テレビ'],
      dtype=object)

In [103]:
# 初期行動の規定
time_table_0 = data_hourly_cleaned[(data_hourly_cleaned["Time"].dt.hour == 0) & (data_hourly_cleaned["Time"].dt.minute == 0)] # 0:00における行為者率を抽出
action_idx = roulette_wheel_selection(time_table_0["Rate"].to_list()) # その時刻における行為者率を重みとしてルーレット選択
Actions =[] # 選択した行為種別の記録
Actions.append(action_idx)
for t in range(4*24): # 15min間隔で24時間分繰り返す。
    transition_r_t = transition_rates_hourly[t] # 時刻tの遷移確率行列を抽出
    action_idx = roulette_wheel_selection(list(transition_r_t[action_idx,:])) # 遷移確率をもとにt+1における行動種別を決定
    Actions.append(action_idx) # 行動種別履歴に追加
acts = middle
actMaster = dict(zip(list(range(len(acts))),acts))
print([actMaster[a] for a in Actions])

['テレビ', 'テレビ', 'テレビ', 'テレビ', '睡眠', '睡眠', '睡眠', '睡眠', '睡眠', '睡眠', '睡眠', '睡眠', '睡眠', '睡眠', '睡眠', '睡眠', '睡眠', '睡眠', '睡眠', '睡眠', '睡眠', '睡眠', '睡眠', '睡眠', '睡眠', '睡眠', '睡眠', '睡眠', '睡眠', 'ラジオ', 'ラジオ', '会話・交際', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', '学業', 'ラジオ', 'ラジオ', 'ラジオ', 'ラジオ', 'ラジオ', 'ラジオ', 'ラジオ', 'ラジオ', 'ラジオ', '家庭雑事', '家庭雑事', '家庭雑事', '家庭雑事', '家庭雑事', '家庭雑事', '家庭雑事', '家庭雑事', '家庭雑事', '家庭雑事', '家庭雑事', '家庭雑事', '新聞', '新聞', '睡眠']


In [100]:
get_P_i(transition_rates_hourly, )

TypeError: get_P_i() missing 2 required positional arguments: 'Transition_rates' and 'Daily_table'